In [40]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import os,shutil,json
import argparse
import copy
import matplotlib.pyplot as plt

from statsmodels.stats.proportion import proportion_confint
import glob
import os
import copy
from scipy.special import comb
parser = argparse.ArgumentParser()
parser.add_argument("-ablation_ratio_test", type=float, default=0.0107)#for randomized_ablation
parser.add_argument("-ablation_ratio_test1", type=float, default=0.0161)#for MMCert
parser.add_argument("-ablation_ratio_test2", type=float, default=0.0054)#for MMCert
parser.add_argument("-r1_r2_ratio", type=int, default=1)#for MMCert
parser.add_argument("-alpha", type=float, default=0.001)
parser.add_argument("-c", type=float, help="number of test samples", default=58)
parser.add_argument("-num_ablated_inputs", type=int, default=100)

def plot_tensor_image(tensor_image, batch_idx=0):
    # If tensor has more than 2 dimensions, select the batch index
    if len(tensor_image.shape) > 2:
        tensor_image = tensor_image[batch_idx]
    
    # Convert tensor to numpy array
    image_array = tensor_image.detach().cpu().numpy()

    # Plot the image
    plt.imshow(image_array, cmap='gray')
    plt.colorbar()
    plt.show()
def _lower_confidence_bound(NA: int, N: int, alpha: float) -> float:
        """ Returns a (1 - alpha) lower confidence bound on a bernoulli proportion.
        This function uses the Clopper-Pearson method.
        :param NA: the number of "successes"
        :param N: the number of total draws
        :param alpha: the confidence level
        :return: a lower bound on the binomial proportion which holds true w.p at least (1 - alpha) over the samples
        """
        return proportion_confint(NA, N, alpha, method="beta")
def get_bound_list(args):
    l =[]
    for i in range(101):
        lower,upper = _lower_confidence_bound(i, args.num_ablated_inputs, args.alpha)
        l.append((lower,upper))
    return l


def get_bounds(args,counts,bound_list):
    lower_bounds = np.zeros((58,375,1242))
    upper_bounds = np.zeros((58,375,1242))
    for k in range(58):
        for i in range(375):
            for j in range(1242):
                lower_bounds[k][i][j],upper_bounds[k][i][j] = bound_list[counts[k][i][j]]
    return lower_bounds, upper_bounds

import numpy as np

def get_bounds1(args, counts, bound_list):
    bound_array = np.array(bound_list)  # Convert list of tuples to 2D numpy array

    # Index into bound_array using counts to fetch the corresponding bounds
    lower_bounds = bound_array[counts][:, :, :, 0]
    upper_bounds = bound_array[counts][:, :, :, 1]

    return lower_bounds, upper_bounds

def get_certified_pixels(all_gt,lower_positive,upper_positive,lower_negative,upper_negative):
    certified_pixels = np.zeros((58,375,1242))
    for k in range(58):
        print(k)
        for i in range(375):
            for j in range(1242):
                if all_gt[k][i][j] ==0:
                    if lower_negative[k][i][j] > upper_positive[k][i][j]:
                        certified_pixels[k][i][j] = 1
                    else:
                        certified_pixels[k][i][j] = 0
                if all_gt[k][i][j] ==1:
                    if lower_positive[k][i][j] > upper_negative[k][i][j]:
                        certified_pixels[k][i][j] = 1
                    else:
                        certified_pixels[k][i][j] = 0
    return certified_pixels

import numpy as np

def get_certified_pixels1(all_gt, lower_positive, upper_positive, lower_negative, upper_negative,e1,e2,n1,n2,k1,k2):
    # Initialize the tensor with zeros
    certified_pixels = np.zeros((58, 375, 1242))
    delta = 1-((comb(e1,k1, exact=True)*comb(e2,k2, exact=True))/(comb(n1,k1, exact=True)*comb(n2,k2, exact=True)))
    lower_positive= np.array(lower_positive)-delta
    upper_positive= np.array(upper_positive)+delta
    lower_negative= np.array(lower_negative)-delta
    upper_negative= np.array(upper_negative)+delta
    # For ground truth values of 0
    mask_gt0 = (all_gt == 0)
    certified_pixels[mask_gt0] = (lower_negative[mask_gt0] > upper_positive[mask_gt0]).astype(np.float32)

    # For ground truth values of 1
    mask_gt1 = (all_gt == 1)
    certified_pixels[mask_gt1] = (lower_positive[mask_gt1] > upper_negative[mask_gt1]).astype(np.float32)
    
    return certified_pixels

    
def calculate_certified_recall(all_gt, certified_pixels):
    # Ensure inputs are torch tensors
    all_gt = torch.tensor(all_gt)
    certified_pixels = torch.tensor(certified_pixels)

    # Calculate True Positives
    TP = torch.sum((all_gt == 1) & (certified_pixels == 1))

    # Calculate False Positives
    FN = torch.sum((all_gt == 1) & (certified_pixels == 0))

    # Handle the case where TP + FP = 0 to avoid division by zero
    if TP + FN == 0:
        return 0.0

    # Calculate precision
    recall = TP.float() / (TP + FN).float()

    return recall.item()  # return as a scalar
def calculate_certified_metrics(all_gt, certified_pixels):
    # Ensure inputs are torch tensors
    all_gt = torch.tensor(all_gt)
    certified_pixels = torch.tensor(certified_pixels)

    # Calculate True Positives
    TP = torch.sum((all_gt == 1) & (certified_pixels == 1))

    # Calculate False Negative
    FN = torch.sum((all_gt == 1) & (certified_pixels == 0))
    # Calculate False Positive
    FP = torch.sum((all_gt == 0) & (certified_pixels == 0))
    # Calculate True Negative
    TN = torch.sum((all_gt == 0) & (certified_pixels == 1))
    # Handle the case where TP + FP = 0 to avoid division by zero

    # Calculate precision
    pixel_accuracy = (TP.float()+TN.float()) / (TP + FN+FP+TN).float()
    recall = TP.float() / (TP + FN).float()
    precision = TP.float() / (TP + FP).float()
    iou = TP.float() / (TP + FP+FN).float()
    return pixel_accuracy.item(), recall.item(), precision.item(), iou.item() # return as a scalar
def calculate_certified_per_pixel_accuracy(all_gt, certified_pixels):
    # Ensure inputs are torch tensors
    all_gt = torch.tensor(all_gt)
    certified_pixels = torch.tensor(certified_pixels)
    

    return certified_pixels.sum().item()/(375*1242)/58  # return as a scalar
def get_alpha_list(args,n1,n2,k1,k2,certification_method="MMCert"):
    l =np.ones((25,101))
    
    for r in range(25):
        r1 = r
        r2 = args.r1_r2_ratio*r
        e1 = n1-r1
        e2 = n2-r2
        if certification_method=="MMCert":
            delta = 1-((comb(e1,k1, exact=True)*comb(e2,k2, exact=True))/(comb(n1,k1, exact=True)*comb(n2,k2, exact=True)))
        else:
            delta = 1-((comb(e1+e2,k1+k2, exact=True))/(comb(n1+n2,k1+k2, exact=True)))
        for i in range(101):
            alpha = 0.00001
            for j in range(50):
                lower_positive,upper_positive = _lower_confidence_bound(i, 100, alpha)
                lower_negative,upper_negative = _lower_confidence_bound(100-i, 100, alpha)
                #print((lower_positive-delta), (upper_negative+delta))
                if ((lower_positive-delta) > (upper_negative+delta)) or ((lower_negative-delta) > (upper_positive+delta)):
                    alpha = alpha/10
                else:
                    if j ==0:
                        l[r][i]=1
                    else:
                        l[r][i]=alpha
                    break
    return l
def get_all_alpha(all_pred,all_gt,alpha_list,r):
    all_alpha = torch.ones(all_pred.shape)
    for i in range(58):
        for j in range(375):
            for k in range(1242):
                count = all_pred[i][j][k]
                if (all_gt[i][j][k]==1 and count<=50) or (all_gt[i][j][k]==0 and count>=50):
                    all_alpha[i][j][k] = 1
                else:
                    all_alpha[i][j][k]= alpha_list[r][count]
    
    return all_alpha

def get_all_alpha1(all_count, all_gt, alpha_list, r):
    all_gt = torch.tensor(all_gt)
    all_count = torch.tensor(all_count)
    # Create a tensor of ones with the same shape as all_count
    all_alpha = torch.ones_like(all_count,dtype=torch.float64)
    alpha_list = torch.tensor(alpha_list,dtype=torch.float64) 
    # Mask for where all_gt is 1 and count in all_pred is <= 50
    mask1 = (all_gt == 1) & (all_count <= 50)
    
    # Mask for where all_gt is 0 and count in all_pred is >= 50
    mask2 = (all_gt == 0) & (all_count >= 50)
    
    # Update all_alpha values to 0 where either mask1 or mask2 is true
    all_alpha[mask1 | mask2] = 1

    # For other values, get from alpha_list
    all_alpha[~(mask1 | mask2)] = alpha_list[r][all_count[~(mask1 | mask2)]]
    
    return all_alpha

def get_certified_pixels_adaptive(all_alpha):
    total_budget = 0.001
    flattened_alpha = all_alpha.view(all_alpha.size(0), -1)

    # Sort the flattened tensor along its last dimension
    sorted_values, sorted_indices = torch.sort(flattened_alpha, dim=-1)

    cumsum_values = torch.cumsum(sorted_values, dim=-1)

    # Find the index where the cumulative sum exceeds the total_budget for each row
    #print((cumsum_values <= total_budget).sum(dim=-1).tolist())
    mask = (cumsum_values <= total_budget)#.sum(dim=-1).tolist()
    B = all_alpha.size(0)
    certified_pixels = torch.zeros_like(flattened_alpha)

    # Using the mask, get the corresponding indices from sorted_indices for each image in the batch
    for i in range(B):
        retained_indices = sorted_indices[i][mask[i]]
        certified_pixels[i].index_fill_(0, retained_indices, 1)
    
    return certified_pixels.view_as(all_alpha)


args = parser.parse_args([])


In [43]:
print("========MSCert=========")
n1 = 375*1242
n2 = 375*1242
k1 = int(n1*args.ablation_ratio_test1)
k2 = int(n2*args.ablation_ratio_test2)
all_outputs = torch.load("output\\"+"MMCert"+"_ablation-ratio-test1="+str(args.ablation_ratio_test1)+"_ablation-ratio-test2="+str(args.ablation_ratio_test2)+"_all_outputs.pth")
all_pred =all_outputs["all_pred"]
all_gt = all_outputs["all_gt"]
all_pred = torch.stack([torch.stack(tensors) for tensors in all_pred])
all_gt = torch.stack([torch.stack(tensors) for tensors in all_gt])
all_pred = torch.transpose(all_pred, 0, 1)
all_gt = all_gt[0]

#print(all_globalacc[0])
#print(torch.min(all_iou[0]),torch.max(all_iou[0]))
#{"all_globalacc":all_globalacc, "all_pre":all_pre, "all_recall":all_recall, "all_F_score":all_F_score, "all_iou":all_iou}
all_count = torch.sum(all_pred, dim=1)
all_predictions = (all_count > 50).int()
print(all_pred.shape)

alpha_list = get_alpha_list(args,n1,n2,k1,k2, certification_method="MMCert")
certified_metrics_mmcert = []
for r in range(25):
    all_alpha = get_all_alpha1(all_count, all_gt, alpha_list,r)
    #print(all_alpha)
    certified_pixels = get_certified_pixels_adaptive(all_alpha)
    certified_metrics_mmcert.append(calculate_certified_metrics(all_gt, certified_pixels))
    #plot_tensor_image(certified_pixels,2)
    print("r, pixel_acc, recall, precision,iou")
    print(r,calculate_certified_metrics(all_gt, certified_pixels))


========MSCert=========
torch.Size([58, 100, 375, 1242])


C:\Users\dongs\AppData\Local\Temp\ipykernel_30532\386582063.py:201: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_gt = torch.tensor(all_gt)
C:\Users\dongs\AppData\Local\Temp\ipykernel_30532\386582063.py:202: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_count = torch.tensor(all_count)
C:\Users\dongs\AppData\Local\Temp\ipykernel_30532\386582063.py:133: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_gt = torch.tensor(all_gt)
C:\Users\dongs\AppData\Local\Temp\ipykernel_30532\386582063.py:134: UserWarning: To copy construct from a 

r, pixel_acc, recall, precision,iou
0 (0.9707775712013245, 0.9100191593170166, 0.9269986152648926, 0.8491644263267517)
r, pixel_acc, recall, precision,iou
1 (0.9700616598129272, 0.9078407883644104, 0.9251534342765808, 0.8457256555557251)
r, pixel_acc, recall, precision,iou
2 (0.9692890048027039, 0.9054640531539917, 0.9231821298599243, 0.8420230150222778)
r, pixel_acc, recall, precision,iou
3 (0.968715488910675, 0.9034517407417297, 0.921931803226471, 0.8392465710639954)
r, pixel_acc, recall, precision,iou
4 (0.9679130911827087, 0.9009279608726501, 0.919928252696991, 0.835416316986084)
r, pixel_acc, recall, precision,iou
5 (0.9668677449226379, 0.8973907828330994, 0.9175270199775696, 0.8304075598716736)
r, pixel_acc, recall, precision,iou
6 (0.9663081169128418, 0.8954989314079285, 0.9162366390228271, 0.8277344107627869)
r, pixel_acc, recall, precision,iou
7 (0.9651195406913757, 0.8915169835090637, 0.9134615659713745, 0.8220841884613037)
r, pixel_acc, recall, precision,iou
8 (0.96447426080

In [ ]:
print("========randomized_ablation=========")
n1 = 375*1242
n2 = 375*1242
k1 = int(n1*args.ablation_ratio_test)
k2 = int(n2*args.ablation_ratio_test)
all_outputs = torch.load("output\\"+"randomized_ablation"+"_ablation-ratio-test="+str(args.ablation_ratio_test)+"_all_outputs.pth")
all_pred =all_outputs["all_pred"]
all_gt = all_outputs["all_gt"]
all_pred = torch.stack([torch.stack(tensors) for tensors in all_pred])
all_gt = torch.stack([torch.stack(tensors) for tensors in all_gt])
all_pred = torch.transpose(all_pred, 0, 1)
all_gt = all_gt[0]

#print(all_globalacc[0])
#print(torch.min(all_iou[0]),torch.max(all_iou[0]))
#{"all_globalacc":all_globalacc, "all_pre":all_pre, "all_recall":all_recall, "all_F_score":all_F_score, "all_iou":all_iou}
all_count = torch.sum(all_pred, dim=1)
all_predictions = (all_count > 50).int()
print(all_pred.shape)

alpha_list = get_alpha_list(args,n1,n2,k1,k2, certification_method="randomized_ablation")
certified_metrics_ra = []
for r in range(25):
    all_alpha = get_all_alpha1(all_count, all_gt, alpha_list,r)
    #print(all_alpha)
    certified_pixels = get_certified_pixels_adaptive(all_alpha)
    #plot_tensor_image(certified_pixels,2)
    certified_metrics_ra.append(calculate_certified_metrics(all_gt, certified_pixels))
    print("r, pixel_acc, recall, precision,iou")
    print(r,calculate_certified_metrics(all_gt, certified_pixels))


========randomized_ablation=========
torch.Size([58, 100, 375, 1242])


C:\Users\dongs\AppData\Local\Temp\ipykernel_30532\386582063.py:201: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_gt = torch.tensor(all_gt)
C:\Users\dongs\AppData\Local\Temp\ipykernel_30532\386582063.py:202: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_count = torch.tensor(all_count)
C:\Users\dongs\AppData\Local\Temp\ipykernel_30532\386582063.py:133: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_gt = torch.tensor(all_gt)
C:\Users\dongs\AppData\Local\Temp\ipykernel_30532\386582063.py:134: UserWarning: To copy construct from a 

r, pixel_acc, recall, precision,iou
0 (0.9668641090393066, 0.8817130327224731, 0.9313355684280396, 0.827894926071167)


In [23]:
print("==========vanilla=========")
positive_counts = all_count
negative_counts = 100- all_count
args.alpha = 0.001/n1
print(positive_counts.shape)
bound_list = get_bound_list(args)
lower_positive, upper_positive = get_bounds1(args,positive_counts,bound_list)
lower_negative, upper_negative = get_bounds1(args,negative_counts,bound_list)
rs = []
CAs = []
CAs_baseline = []
RANGE=30
for r in range(RANGE):
    r1 = r
    r2 = args.r1_r2_ratio*r
    e1 = n1-r1
    e2 = n2-r2
    certified_pixels= get_certified_pixels1(all_gt,lower_positive,upper_positive,lower_negative,upper_negative,e1,e2,n1,n2,k1,k2)
    print(r,calculate_certified_recall(all_gt, certified_pixels))
    ca = certified_pixels.sum().item()/(375*1242)/58
    #print(r, ca)
    CAs.append(ca)
    rs.append(r)

#certified_pixels= get_certified_pixels1(all_gt,lower_positive,upper_positive,lower_negative,upper_negative,e1,e2,n1,n2,k1,k2)

==========vanilla=========
torch.Size([58, 375, 1242])


C:\Users\dongs\AppData\Local\Temp\ipykernel_30532\78833124.py:101: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_gt = torch.tensor(all_gt)


0 0.9015240669250488
1 0.8980993032455444
2 0.8944099545478821
3 0.8924378156661987
4 0.8882236480712891
5 0.8859847187995911
6 0.8811527490615845
7 0.8785921335220337
8 0.8758500218391418
9 0.8693690896034241
10 0.8654698729515076
11 0.8609969019889832
12 0.8552723526954651
13 0.8472498655319214
14 0.8472498655319214
15 0.8353611826896667
16 0.813547670841217
17 0.813547670841217
18 0.0
19 0.0
20 0.0
21 0.0
22 0.0
23 0.0
24 0.0
25 0.0
26 0.0
27 0.0
28 0.0
29 0.0


In [38]:
print(_lower_confidence_bound(5, 100, 0.001))
print(_lower_confidence_bound(95, 100, 0.001))

(0.006433946195449594, 0.16361499252741493)
(0.836385007472585, 0.9935660538045504)


In [73]:
print(certified_pixels.sum().item()/(58*375*1242))

0.9689200214707461


In [74]:
print(lower_positive)

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
